In [109]:
#Import necessary libraries
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [108]:
# Load the data from each file
with open('data1') as f:
    data = json.load(f)
#with open('data2') as f:
    #data2 = json.load(f)
#with open('data3') as f:
    #data3 = json.load(f)
#with open('data4') as f:
    #data4 = json.load(f)
#data = {}
#data.update(data1)
#data.update(data2)
#data.update(data3)
#data.update(data4)

# Exploration and Preprocessing Data

In [110]:
print(len(data.keys()))

5918


In [111]:
#extract all paper ids
paper_id=data.keys()
paper_id

dict_keys(['1009.4013', '1009.4019', '1009.4046', '1009.4091', '1009.4102', '1009.4188', '1009.4214', '1009.4219', '1009.4269', '1009.4300', '1009.4330', '1009.4352', '1009.4375', '1009.4386', '1009.4400', '1009.4471', '1009.4489', '1009.4499', '1009.4503', '1009.4529', '1009.4597', '1009.4625', '1009.4647', '1009.4672', '1009.4766', '1009.4780', '1009.4787', '1009.4798', '1009.4823', '1009.4825', '1009.4877', '1009.4880', '1009.4954', '1009.4970', '1009.4975', '1009.4995', '1009.5012', '1009.5019', '1009.5026', '1009.5055', '1009.5104', '1009.5108', '1009.5145', '1009.5146', '1009.5158', '1009.5161', '1009.5167', '1009.5168', '1009.5183', '1009.5227', '1009.5233', '1009.5268', '1009.5282', '1009.5321', '1009.5432', '1009.5435', '1009.5473', '1009.5588', '1009.5614', '1009.5621', '1009.5625', '1009.5626', '1009.5628', '1009.5696', '1009.5734', '1009.5750', '1009.5758', '1009.5760', '1009.5761', '1009.5764', '1009.5791', '1009.5802', '1009.5829', '1009.5863', '1009.5878', '1009.5900', '

In [5]:
data["1101.1597"]["authors"]

'bernd sturmfels,volkmar welker'

In [6]:
#Check for need of preprocssing text
data["1101.1597"]["abstract"]

'model statistical ranking family probability distributions whose states orderings fixed finite set items  represent orderings maximal chains graded poset  widely used ranking models parameterized rational function model parameters  define algebraic varieties  study varieties perspective combinatorial commutative algebra  one models  plackettluce model  nontoric  five others toric  birkhoff model  ascending model  csiszar model  inversion model  bradleyterry model  models examine toric algebra  lattice polytope  markov basis '

In [7]:
#Get All Categrories in papers
all_categories = set()

# iterate through all papers
for paper_id, paper_data in data.items():
    # get the category of the current paper and add it to the set of all categories
    category = paper_data.get('category')
    if category:
        all_categories.add(category)

# print all categories
for category in all_categories:
    print(category)

cs.SY,math.OC,q-bio.QM
physics.data-an,cond-mat.stat-mech,cs.LG,q-bio.NC,q-bio.QM
stat.ML,cs.AI,cs.LG
math.AG,cs.CV
cs.AI,cs.IR
cs.NE,cs.CV
math.PR,cs.LG,stat.ML
cs.GT,cs.AI
cs.SY,cs.AR,cs.SE
cs.IT,math.IT,math.ST,stat.TH
cs.CV,cs.NI
cond-mat.dis-nn,cond-mat.stat-mech,cs.IT,cs.LG,math.IT
cs.NI,cs.CL
cs.DL,physics.soc-ph
cs.SD,cs.MM
math.FA,cs.IT,cs.NA,math.IT
cs.LG,cs.SY,math.OC
cs.DC,cs.CR,cs.LG
cs.AI,cs.CR,math.DS,math.GN
cs.DC,math.CO,math.GR
cs.CV,cs.DS
cs.LG,stat.ML
physics.data-an,astro-ph.HE,cs.CE,nlin.CD
math.CO,cs.CR,math.OC
cs.AI,cs.CC,cs.DB
stat.AP,cs.IT,math.IT
cs.CL,math.CT
stat.ML,cs.AI
physics.soc-ph,cond-mat.stat-mech,cs.SI,q-bio.PE
cs.SC,hep-ph
nlin.CG,cs.NE
cs.CY,cs.SI,physics.soc-ph
cs.DB,cs.LO
math.OC,cs.NA
math.NA,cs.DC
cs.DM,cs.CV,math.OC
q-bio.QM,cs.CE
cs.GT,cs.IT,cs.SY,math.IT,math.OC
physics.soc-ph,cs.SI,nlin.AO
cs.AI,cs.MS
cs.DS,cs.CC,cs.SC
cs.MS,cs.LO
cs.IR,cs.DB,cs.LG
physics.soc-ph,cond-mat.stat-mech,cs.SI
math.CO,cs.DM,cs.GT
cs.SC,math.CO
cs.DL,cs.IR
cs.CC

In [8]:
len(all_categories)

1736

## Convert Data to Dataframe 

In [15]:
# Create a list of dictionaries where each dictionary corresponds to one article
articles = []
for paper_id, values in data.items():
    article = {'paper_id': paper_id,
               'abstract': values['abstract'],
               'title': values['title'],
               'venue': values['venue'],
               'category': values['category']}
    articles.append(article)

# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(articles)

In [89]:
#check for duplicates
duplicates = df.duplicated()
num_duplicates = duplicates.sum()
print("Number of duplicates:", num_duplicates)

Number of duplicates: 0


In [94]:
df.isnull().sum()

paper_id    0
abstract    0
title       0
venue       0
category    0
dtype: int64

In [97]:
df.head(5)

,paper_id,abstract,title,venue,category
0,1009.4013,many firms days opting specialize rather gener...,an analysis of transaction and joint-patent ap...,CoRR,0
1,1009.4019,paper analyses changes public opinion tracking...,emotional reactions and the pulse of public op...,CoRR,1
2,1009.4046,randomaccess networks ieee 80211 network dif...,channel-coded collision resolution by exploiti...,ICC,2
3,1009.4091,mimo wireless channel offers rich ground quali...,on the flow-level delay of a spatial multiplex...,ICC,3
4,1009.4102,article presents new search algorithm nphard p...,the lazy flipper: map inference in higher-orde...,CoRR,4


In [98]:
df.describe

<bound method NDFrame.describe of        paper_id                                           abstract  \
0     1009.4013  many firms days opting specialize rather gener...   
1     1009.4019  paper analyses changes public opinion tracking...   
2     1009.4046  randomaccess networks  ieee 80211 network  dif...   
3     1009.4091  mimo wireless channel offers rich ground quali...   
4     1009.4102  article presents new search algorithm nphard p...   
...         ...                                                ...   
5913  1206.1579  generalized traveling salesman problem  gtsp  ...   
5914  1206.1630  address issue generating cutting planes mixed ...   
5915  1206.1633  investigate use linear programming tools solvi...   
5916  1206.1634  investigate downlink scheduling problem markov...   
5917  1206.1698  quadrangulation graph embedded sphere face bou...   

                                                  title  \
0     an analysis of transaction and joint-patent ap...   
1     e

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5918 entries, 0 to 5917
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   paper_id  5918 non-null   object
 1   abstract  5918 non-null   object
 2   title     5918 non-null   object
 3   venue     5918 non-null   object
 4   category  5918 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 231.3+ KB


## Tokenize Text

In [112]:
def preprocess_abstract(abstract):
    # Convert to lowercase
    abstract = abstract.lower()
    
    # Remove punctuation
    abstract = re.sub(r'[^\w\s]', '', abstract)
    
    # Tokenize words
    words = word_tokenize(abstract)
    
    # Remove stop words
    words = [w for w in words if w not in stopwords.words('english')]
    
    # Join words back into a string
    processed_abstract = ' '.join(words)
    
    return processed_abstract

In [114]:
abstract= data['1010.2989']['abstract']
abstract1_after=preprocess_abstract(abstract)
abstract1_after

'total coloring graph coloring vertices edges adjacent vertices edges incident vertices edges obtain color graph total coloring colors least one vertex edge colored edges incident vertex together colored consecutive colors degree vertex paper investigate properties interval total colorings also determine exact values least greatest possible number colors colorings classes graphs'

# Recommend based on specific Category

In [9]:
# create a dictionary of category to paper ids
category_to_ids = {}
for paper_id, paper_data in data.items():
    category = paper_data['category']
    if category in category_to_ids:
        category_to_ids[category].append(paper_id)
    else:
        category_to_ids[category] = [paper_id]

# define a function to recommend papers based on category
def recommend_papers(category):
    if category not in category_to_ids:
        return []
    paper_ids = category_to_ids[category]
    return [data[paper_id] for paper_id in paper_ids]



In [10]:
#CS.AI Category Papers
recommended_papers = recommend_papers('cs.AI')
for paper in recommended_papers:
    print(paper['title'])

general scaled support vector machines
hierarchical multiclass decompositions with application to authorship determination
a protocol for self-synchronized duty-cycling in sensor networks: generic implementation in wiselib
a partial taxonomy of substitutability and interchangeability
a unifying probabilistic perspective for spectral dimensionality reduction: insights and new models
qualitative reasoning about relative direction on adjustable levels of granularity
reasoning about cardinal directions between extended objects: the hardness result
detecting ontological conflicts in protocols between semantic web services
distributed graph coloring: an approach based on the calling behavior of japanese tree frogs
ursa: a system for uniform reduction to sat
a study on the relation between linguistics-oriented and domain-specific semantics
using semantic wikis for structured argument in medical domain
creating a new ontology: a modular approach
a semantic approach for the requirement-driven d

# Content based Cosine Similarity

In [11]:
#recommendation system based on cosine similarity

# Vectorize the data
abstracts = [data[key]['abstract'] for key in data]
titles = [data[key]['title'] for key in data]
categories = [data[key]['category'] for key in data]
venues = [data[key]['venue'] for key in data]



vectorizer = TfidfVectorizer(stop_words='english')
tfidf_abstracts = vectorizer.fit_transform(abstracts)
tfidf_titles = vectorizer.fit_transform(titles)
tfidf_categories = vectorizer.fit_transform(categories)
tfidf_venues = vectorizer.fit_transform(venues)



# Calculate cosine similarity
cos_sim_abstracts = cosine_similarity(tfidf_abstracts)
cos_sim_titles = cosine_similarity(tfidf_titles)
cos_sim_categories = cosine_similarity(tfidf_categories)
cos_sim_venues = cosine_similarity(tfidf_venues)

# Combine the cosine similarity scores
cos_sim_total = (cos_sim_abstracts + cos_sim_titles + cos_sim_categories + cos_sim_venues) / 4

# Define a function to get the top N similar papers
def get_similar_papers(paper_id, N=5):
    # Get the index of the paper in the dataset
    print('Your Paper\'s Title is:\n', data[paper_id]['title'])
    print('Your Paper\'s Abstract is:\n', data[paper_id]['abstract'])
    idx = list(data.keys()).index(paper_id)
    # Get the cosine similarities of the paper with all other papers
    sim_scores = cos_sim_total[idx]
    # Get the indices of the top N similar papers
    top_indices = sim_scores.argsort()[::-1][1:N+1]
    # Get the paper ids of the top N similar papers
    top_paper_ids = [list(data.keys())[i] for i in top_indices]
    #get title of paper
    top_titles = [data[key]['title'] for key in top_paper_ids]
    # Get the abstracts of the top N similar papers
    top_abstracts = [data[key]['abstract'] for key in top_paper_ids]
    top_category=[data[key]['category'] for key in top_paper_ids]
    # Return the paper ids and abstracts of the top N similar papers
    return list(zip(top_paper_ids,top_titles,top_category, top_abstracts))

In [12]:
get_similar_papers('1009.5026', N=10)

Your Paper's Title is:
 on the fictitious play and channel selection games
Your Paper's Abstract is:
 considering interaction mutual interference different radio devices  channel selection  cs  problem decentralized parallel multiple access channels modeled strategicform games   show cs problem potential game  pg  thus fictitious play  fp  converges nash equilibrium  ne  either pure mixed strategies  using 2player 2channel game  shown convergence mixed strategies might lead cycles action profiles lead individual spectral efficiencies  se  worse se worst ne mixed pure strategies  finally  exploiting fact cs problem pg aggregation game  present method implement fp local information minimum feedback 


[('1010.0344',
  'alternating-offer bargaining games over the gaussian interference channel',
  'cs.IT,cs.GT,math.IT',
  'paper tackles problem two selfish users jointly determine operating point achievable rate region twouser gaussian interference channel bargaining  previous work  incentive conditions two users cooperate using simple version hankobayashi scheme studied nash bargaining solution  nbs  used obtain fair operating point  noncooperative bargaining game alternating offers adopted model bargaining process rates resulting equilibrium outcome analyzed  particular  shown operating point resulting formulated bargaining game depends cost delay bargaining bargaining proceeds  associated bargaining problem regular  unique perfect equilibrium exists lies individual rational efficient frontier achievable rate region  besides  equilibrium outcome approaches nbs bargaining costs users negligible '),
 ('1010.2439',
  'conservation law of utility and equilibria in non-zero sum games',
  

# KNN Algorithm

In [13]:

# Combine the vectorized data into one feature matrix
X = np.hstack((tfidf_abstracts.toarray(), tfidf_titles.toarray(), tfidf_categories.toarray(), tfidf_venues.toarray()))

# Define the KNN model and fit the feature matrix
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(X)

# Define a function to get the top N similar papers
def get_similar_papers_knn(paper_id, N=10):
    # Get the index of the paper in the dataset
    print('Your Paper\'s Title is:\n', data[paper_id]['title'])
    print('Your Paper\'s Abstract is:\n', data[paper_id]['abstract'])
    idx = list(data.keys()).index(paper_id)
    # Get the K nearest neighbors of the paper
    X_input = X[idx].reshape(1,-1)
    distances, indices = model_knn.kneighbors(X_input, n_neighbors=N+1)
    # Get the paper ids of the K nearest neighbors
    similar_paper_ids = [list(data.keys())[i] for i in indices.flatten() if i != idx]
    #get title of paper
    top_titles = [data[key]['title'] for key in similar_paper_ids]
    # Get the abstracts of the K nearest neighbors
    top_abstracts = [data[key]['abstract'] for key in similar_paper_ids]
    top_category=[data[key]['category'] for key in similar_paper_ids]
    # Return the paper ids and abstracts of the top N similar papers
    return list(zip(similar_paper_ids,top_titles,top_category, top_abstracts))


In [14]:
get_similar_papers_knn('1009.5026',N=10)

Your Paper's Title is:
 on the fictitious play and channel selection games
Your Paper's Abstract is:
 considering interaction mutual interference different radio devices  channel selection  cs  problem decentralized parallel multiple access channels modeled strategicform games   show cs problem potential game  pg  thus fictitious play  fp  converges nash equilibrium  ne  either pure mixed strategies  using 2player 2channel game  shown convergence mixed strategies might lead cycles action profiles lead individual spectral efficiencies  se  worse se worst ne mixed pure strategies  finally  exploiting fact cs problem pg aggregation game  present method implement fp local information minimum feedback 


[('1010.0344',
  'alternating-offer bargaining games over the gaussian interference channel',
  'cs.IT,cs.GT,math.IT',
  'paper tackles problem two selfish users jointly determine operating point achievable rate region twouser gaussian interference channel bargaining  previous work  incentive conditions two users cooperate using simple version hankobayashi scheme studied nash bargaining solution  nbs  used obtain fair operating point  noncooperative bargaining game alternating offers adopted model bargaining process rates resulting equilibrium outcome analyzed  particular  shown operating point resulting formulated bargaining game depends cost delay bargaining bargaining proceeds  associated bargaining problem regular  unique perfect equilibrium exists lies individual rational efficient frontier achievable rate region  besides  equilibrium outcome approaches nbs bargaining costs users negligible '),
 ('1010.2439',
  'conservation law of utility and equilibria in non-zero sum games',
  

# Decision Tree

In [19]:
# Convert text features to vectors using CountVectorizer
vectorizer = CountVectorizer()
X_abstract = vectorizer.fit_transform(df['abstract'])
X_title = vectorizer.fit_transform(df['title'])
X_venue = vectorizer.fit_transform(df['venue'])

# Combine the vectorized features into a single feature matrix
X = pd.concat([pd.DataFrame(X_abstract.toarray()), 
               pd.DataFrame(X_title.toarray()), 
               pd.DataFrame(X_venue.toarray())], axis=1)

# Convert category feature to numerical values
df['category'] = pd.factorize(df['category'])[0]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df['paper_id'], test_size=0.2, random_state=42)

# Train decision tree classifier
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [20]:
# Predict recommendations for test set
y_pred = clf.predict(X_test)

In [21]:
predicted_papers=y_pred[:10]
predicted_papers

array(['1106.1957', '1012.1651', '1011.5314', '1010.6255', '1104.4905',
       '1203.4135', '1108.0065', '1104.1080', '1104.4601', '1203.3368'],
      dtype=object)

In [22]:
df.loc[df['paper_id'].isin(predicted_papers), ['title']]

,title
338,on the capacity of the 2-user gaussian mac int...
525,"ml(n)bicgstab: reformulation, analysis and imp..."
639,the rule responder escience infrastructure
1683,approximating the balanced minimum evolution p...
1836,chemxseer digital library gaussian search
1851,inner approximations for polynomial matrix ine...
2261,interdefinability of defeasible logic and logi...
2729,approximating the permanent with fractional be...
5032,between arrow and gibbard-satterthwaite; a rep...
5061,metadata management in scientific computing


# K-means Algorithm

In [66]:
# Extract abstracts from the data
abstracts = [paper["abstract"] for paper in data.values()]

#Feature extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(abstracts)

# K-means clustering
k = 5  # Number of clusters
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X)

# Recommendation generation
def kmeans_recommendations(paper_id, k):
    # Find the cluster of the given paper
    abstract_index = list(data.keys()).index(paper_id)
    cluster_label = kmeans.labels_[abstract_index]

    # Find all papers in the same cluster
    cluster_papers = [paper for idx, paper in enumerate(data.values()) if kmeans.labels_[idx] == cluster_label]

    # Remove the given paper from the cluster
    cluster_papers.remove(data[paper_id])

    # Get the top-k recommended papers from the cluster
    recommendations = cluster_papers[:k]

    return recommendations


In [70]:
paper_ids = data["1009.5026"]
paper_id="1009.5026"
print("Title: {}\nCategory: {}\nAbstract: {}\nVenue: {}".format(paper_ids["title"], paper_ids["category"], paper_ids["abstract"], paper_ids["venue"]))
recommendations = get_recommendations(paper_id, k=10)
print("Recommended Papers:")
for paper in recommendations:
    print("Title:", paper["title"])
    print("Abstract:", paper["abstract"])
    print("Category:", paper["category"])
    print("Venue:", paper["venue"])
    print()


Title: on the fictitious play and channel selection games
Category: cs.GT,cs.IT,math.IT
Abstract: considering interaction mutual interference different radio devices  channel selection  cs  problem decentralized parallel multiple access channels modeled strategicform games   show cs problem potential game  pg  thus fictitious play  fp  converges nash equilibrium  ne  either pure mixed strategies  using 2player 2channel game  shown convergence mixed strategies might lead cycles action profiles lead individual spectral efficiencies  se  worse se worst ne mixed pure strategies  finally  exploiting fact cs problem pg aggregation game  present method implement fp local information minimum feedback 
Venue: CoRR
Recommended Papers:
Title: emotional reactions and the pulse of public opinion: measuring the impact of political events on the sentiment of online discussions
Abstract: paper analyses changes public opinion tracking political discussions people voluntarily engage online  unlike polls

# Max factorization

In [60]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF

# Convert text features to vectors using CountVectorizer
vectorizer = CountVectorizer()
X_abstract = vectorizer.fit_transform(df['abstract'])
X_title = vectorizer.fit_transform(df['title'])
X_venue = vectorizer.fit_transform(df['venue'])

# Combine the vectorized features into a single feature matrix
X = np.hstack((X_abstract.toarray(), X_title.toarray(), X_venue.toarray()))

# Convert category feature to numerical values
df['category'] = pd.factorize(df['category'])[0]

# Create a sparse matrix from the feature matrix
X_sparse = csr_matrix(X)

# Apply matrix factorization using Non-Negative Matrix Factorization (NMF)
k = 50  # Number of latent factors
nmf_model = NMF(n_components=k)
W = nmf_model.fit_transform(X_sparse)
H = nmf_model.components_

# Compute the reconstructed matrix
X_reconstructed = np.dot(W, H)

# Calculate the similarity matrix using cosine similarity
similarity_matrix = cosine_similarity(X_reconstructed)

# Define a function to get similar papers based on a given paper ID
def get_similar_papers(paper_id, top_k=5):
    paper_id = df.loc[df['paper_id'] == paper_id, 'paper_id'].values[0]
    paper_index = df[df['paper_id'] == paper_id].index[0]
    similar_papers = np.argsort(similarity_matrix[paper_index])[::-1][:top_k]
    return df.loc[similar_papers, 'paper_id']




C:\Users\meme5\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


In [61]:
similar_papers = get_similar_papers('1009.5026', top_k=10)
print(similar_papers)

38      1009.5026
1829    1104.4433
5221    1204.0908
5189    1204.0166
5574    1205.0856
340     1011.0027
738     1012.3875
3570    1110.6084
995     1101.4849
4873    1203.0224
Name: paper_id, dtype: object


In [62]:
similar_paper_ids = df.loc[df['paper_id'].isin(similar_papers), 'paper_id'].values
similar_paper_titles = df.loc[df['paper_id'].isin(similar_papers), 'title'].values
similar_papers = list(zip(similar_paper_ids, similar_paper_titles))
pd.DataFrame(similar_papers)

,0,1
0,1009.5026,on the fictitious play and channel selection g...
1,1011.0027,joint scheduling and resource allocation in th...
2,1012.3875,optimal and robust transmit designs for miso c...
3,1101.4849,a maximum entropy solution of the covariance e...
4,1104.4433,arc-preserving subsequences of arc-annotated s...
5,1110.6084,the multi-armed bandit problem with covariates
6,1203.0224,label cover instances with large girth and the...
7,1204.0166,worst-case robust multiuser transmit beamformi...
8,1204.0908,a procedural framework and mathematical analys...
9,1205.0856,global optimal solution to discrete value sele...


Conclusion: The Cosine Similarity Algorithm has the best results with KNN since it uses the cosine similarity metric